<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# OpenAI Assistant with Code Interpreter - Exercise Solution

## TASK:

**Create an Assistant that can take in a long text description of a bill at a restuarant and who ordered what and how much items cost, and then split the bill among the party accurately.**


**EXAMPLE INPUT:**

    Mary ordered a burger, fries and coke. I ordered just a salad and coke.
    A burger is $10, fries are $5, a salad is $8, and cokes are $2 each.
    We want to leave an 18% tip. Please split the bill between Mary and me, how much do we each owe?


**EXPECTED OUTPUT:**

    To split the bill, we need to calculate the total cost of the food and drinks, add the 18% tip, and then divide the total by 2 to get the amount each person owes.

    Based on the given prices, Mary ordered a burger ($10), fries ($5), and a coke ($2). So the total cost of her order is $10 + $5 + $2 = $17.

    You ordered a salad ($8) and a coke ($2). So the total cost of your order is $8 + $2 = $10.

    The total bill for both of you is $17 + $10 = $27.

    Now we need to add an 18% tip to the total bill. The tip amount is 18% of $27, which is (18/100) * $27 = $4.86.

    Finally, we can split the total bill including the tip between the two of you.


    Each person owes $15.93.

**Note: We will continue the conversation with this added message input:**

    **Do not split the bill evenly. Have each person only pay for what they ordered, but don't forget the tip.**


**Then get an output something like:**

    To split the bill based on what each person ordered, we can calculate the total cost of Mary's order, the total cost of your order, and then add the 18% tip to the total bill.

    Mary's total cost is $17, and your total cost is $10. 

    The total bill for both of you is $17 + $10 = $27.

    Now we need to add an 18% tip to the total bill. The tip amount is 18% of $27, which is (18/100) * $27 = $4.86.

    Finally, we can add the tip to the total bill and split it between the two of you based on what each person ordered.


    Considering that Mary ordered a burger, fries, and a coke with a total cost of $17, while you ordered a salad and a coke with a total cost of $10, the bill split would be as follows:

    Mary owes $20.06, and you owe $11.80.
    
    
**Complete the tasks below to complete the exercise!**

**TASK: Create a client connection to OpenAI**

In [4]:
# CODE HERE

In [5]:
from openai import OpenAI

In [6]:
client = OpenAI()

**TASK: Create the assistant. Make sure your instructions are clear!**

In [7]:
#CODE HERE

In [8]:
assistant = client.beta.assistants.create(
    name="Bill Splitter Bot",
    instructions="You use Python code to help calculate the correct way to split a bill.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo"
)

**TASK: Initiate a new thread.**

In [10]:
#CODE HERE

In [9]:
thread = client.beta.threads.create()

**TASK: Add a new user message to the thread. Use the string below as the content:**

In [11]:
content = '''Mary ordered a burger, fries and coke. I ordered just a salad and coke.
A burger is $10, fries are $5, a salad is $8, and cokes are $2 each.
We want to leave an 18% tip. Please split the bill between Mary and me, how much do we each owe?'''

In [12]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=content
)

**TASK: Run the thread using the assistant created earlier.**

In [13]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Help itemize and split the bill between party members."
)

In [14]:
import time

while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )
    
    print(run.status)
    
    time.sleep(5)

in_progress
in_progress
in_progress
completed


**TASK: Display the message conversation between the user and the assistant.**

In [15]:
#CODE HERE

In [16]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

In [17]:
def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')

In [18]:
display_thread_messages(messages)

Mary ordered a burger, fries and coke. I ordered just a salad and coke.
A burger is $10, fries are $5, a salad is $8, and cokes are $2 each.
We want to leave an 18% tip. Please split the bill between Mary and me, how much do we each owe?


To split the bill, we need to calculate the total cost of the food and drinks, add the 18% tip, and then divide the total by 2 to get the amount each person owes.

Based on the given prices, Mary ordered a burger ($10), fries ($5), and a coke ($2). So the total cost of her order is $10 + $5 + $2 = $17.

You ordered a salad ($8) and a coke ($2). So the total cost of your order is $8 + $2 = $10.

The total bill for both of you is $17 + $10 = $27.

Now we need to add an 18% tip to the total bill. The tip amount is 18% of $27, which is (18/100) * $27 = $4.86.

Finally, we can split the total bill including the tip between the two of you.


Each person owes $15.93.




**TASK: If the Assistant bot split the bill evenly, add a new message asking the bill to be split based on what people ordered. (e.g. Mary should pay more since she ordered more). Then re-run the thread and re-display all the messages.**

In [19]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", 
    content="Do not split the bill evenly. Have each person only pay for what they ordered, but don't forget the tip."
)

In [20]:
# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [24]:
run.status

'completed'

In [25]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)
display_thread_messages(messages)

Mary ordered a burger, fries and coke. I ordered just a salad and coke.
A burger is $10, fries are $5, a salad is $8, and cokes are $2 each.
We want to leave an 18% tip. Please split the bill between Mary and me, how much do we each owe?


To split the bill, we need to calculate the total cost of the food and drinks, add the 18% tip, and then divide the total by 2 to get the amount each person owes.

Based on the given prices, Mary ordered a burger ($10), fries ($5), and a coke ($2). So the total cost of her order is $10 + $5 + $2 = $17.

You ordered a salad ($8) and a coke ($2). So the total cost of your order is $8 + $2 = $10.

The total bill for both of you is $17 + $10 = $27.

Now we need to add an 18% tip to the total bill. The tip amount is 18% of $27, which is (18/100) * $27 = $4.86.

Finally, we can split the total bill including the tip between the two of you.


Each person owes $15.93.


Do not split the bill evenly. Have each person only pay for what they ordered, but don't 

**TASK: Delete your assistant.**

In [ ]:
# CODE HERE

In [26]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_3KpWQk5VHxedDK3l8wKlI0W9', created_at=1701931629, description=None, file_ids=[], instructions='You use Python code to help calculate the correct way to split a bill.', metadata={}, model='gpt-3.5-turbo', name='Bill Splitter Bot', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])]


In [27]:
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_3KpWQk5VHxedDK3l8wKlI0W9', deleted=True, object='assistant.deleted')


---